<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

## Import related packages to create camera instances

In [1]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import PID
camera = Camera.instance(width=720, height=720)

RuntimeError: Could not initialize camera.  Please see error trace.

## Create related control variables

In [ ]:
global color_x, color_y, color_radius
color_x = color_y = color_radius = 0
global target_valuex
target_valuex = 2100
global target_valuey
target_valuey = 2048

## Create an array that stores HSV color gamut color classification data

In [3]:
import numpy as np
global color_lower
color_lower=np.array([156,43,46])
global color_upperv
color_upper = np.array([180, 255, 255])

## Creating a PID Control Instance

In [16]:
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

## Create a PTZ servo engine instance

In [17]:
from servoserial import ServoSerial
servo_device = ServoSerial() 

serial Open!
serial Close!


## Create a display control

In [18]:
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
color_image = widgets.Image(format='jpeg', width=300, height=300)
display(color_image)

Image(value=b'', format='jpeg', height='300', width='300')

### Color recognition default array data

In [19]:
color_lower=np.array([156,43,46])
color_upper = np.array([180, 255, 255])
target_valuex = target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify red array data

In [9]:
color_lower=np.array([0,43,46])
color_upper = np.array([10, 255, 255])
target_valuex = target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify yellow array data

In [10]:
color_lower=np.array([26,43,46])
color_upper = np.array([34, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify blue array data

In [11]:
color_lower=np.array([100,43,46])
color_upper = np.array([124, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify green array data

In [12]:
color_lower=np.array([35,43,46])
color_upper = np.array([77, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


### Set to identify orange array data

In [13]:
color_lower=np.array([11,43,46])
color_upper = np.array([25, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

27
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x00\x00\n\x02\x08\x00\x00\n\x1b'


## Main process of PTZ movement

In [21]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    frame_=cv2.GaussianBlur(frame,(5,5),0)                    
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,color_lower,color_upper)  
    mask=cv2.erode(mask,None,iterations=2)
    mask=cv2.dilate(mask,None,iterations=2)
    mask=cv2.GaussianBlur(mask,(3,3),0)     
    cnts=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2] 
    if len(cnts)>0:
        cnt = max (cnts,key=cv2.contourArea)
        (color_x,color_y),color_radius=cv2.minEnclosingCircle(cnt)
        if color_radius > 10:
            # Mark the detected color
            cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  
            #Proportion-Integration-Differentiation
            xservo_pid.SystemOutput = color_x
            xservo_pid.SetStepSignal(150)
            xservo_pid.SetInertiaTime(0.01, 0.006)
            target_valuex = int(2048+xservo_pid.SystemOutput)
            # Input Y axis direction parameter PID control input
            yservo_pid.SystemOutput = color_y
            yservo_pid.SetStepSignal(150)
            yservo_pid.SetInertiaTime(0.01, 0.006)
            target_valuey = int(2048+yservo_pid.SystemOutput)
             # Rotate the gimbal to the PID adjustment position
            servo_device.Servo_serial_double_control( 1, target_valuex, 2, target_valuey)
     # Real-time return of image data for display
    color_image.value = bgr8_to_jpeg(frame)

KeyboardInterrupt: 

In [ ]:
camera.stop()